In [7]:
import pandas as pd
import zipfile
import re
import sqlite3
from sqlalchemy import create_engine

# Create a SQLite database engine
database_name = "march_madness.sqlite"
engine = create_engine(f'sqlite:///{database_name}')

dataset_name = "march-machine-learning-mania-2025"
dataframes = {}
zip_filename = dataset_name + '.zip'

try:
    with zipfile.ZipFile(zip_filename, 'r') as zip_file:
        print("Files in the ZIP:", zip_file.namelist())
        
        for filename in zip_file.namelist():
            if filename.endswith('.csv'):
                # Create a clean table name
                table_name = filename.replace('.csv', '').replace(' _ ', '_').replace(' ', '_')
                table_name = re.sub(r'[^a-zA-Z0-9_]', '', table_name)  # Remove any special characters
                
                try:
                    # Special handling for TeamSpellings files
                    if 'TeamSpellings' in filename:
                        df = pd.read_csv(
                            zip_file.open(filename), 
                            encoding='latin1',
                            low_memory=False
                        )
                    else:
                        # Default UTF-8 encoding for other files
                        df = pd.read_csv(
                            zip_file.open(filename), 
                            low_memory=False
                        )
                    
                    # Write the dataframe to SQLite database
                    df.to_sql(table_name, engine, if_exists='replace', index=False)
                    print(f"Loaded and converted to SQL: {filename} -> {table_name}")
                    
                except Exception as e:
                    print(f"Error processing {filename}: {str(e)}")
                    
except FileNotFoundError:
    print(f"Could not find the zip file: {zip_filename}")
except zipfile.BadZipFile:
    print(f"The file {zip_filename} is not a valid zip file")
except Exception as e:
    print(f"An error occurred: {str(e)}")

# Example of how to query the database and convert to pandas dataframe
def query_to_dataframe(query):
    """
    Execute a SQL query and return the results as a pandas DataFrame
    """
    try:
        return pd.read_sql_query(query, engine)
    except Exception as e:
        print(f"Error executing query: {str(e)}")
        return None

# Example usage:
# df = query_to_dataframe("SELECT * FROM TableName LIMIT 5")


Files in the ZIP: ['Cities.csv', 'Conferences.csv', 'MConferenceTourneyGames.csv', 'MGameCities.csv', 'MMasseyOrdinals.csv', 'MNCAATourneyCompactResults.csv', 'MNCAATourneyDetailedResults.csv', 'MNCAATourneySeedRoundSlots.csv', 'MNCAATourneySeeds.csv', 'MNCAATourneySlots.csv', 'MRegularSeasonCompactResults.csv', 'MRegularSeasonDetailedResults.csv', 'MSeasons.csv', 'MSecondaryTourneyCompactResults.csv', 'MSecondaryTourneyTeams.csv', 'MTeamCoaches.csv', 'MTeamConferences.csv', 'MTeamSpellings.csv', 'MTeams.csv', 'SampleSubmissionStage1.csv', 'SeedBenchmarkStage1.csv', 'WConferenceTourneyGames.csv', 'WGameCities.csv', 'WNCAATourneyCompactResults.csv', 'WNCAATourneyDetailedResults.csv', 'WNCAATourneySeeds.csv', 'WNCAATourneySlots.csv', 'WRegularSeasonCompactResults.csv', 'WRegularSeasonDetailedResults.csv', 'WSeasons.csv', 'WSecondaryTourneyCompactResults.csv', 'WSecondaryTourneyTeams.csv', 'WTeamConferences.csv', 'WTeamSpellings.csv', 'WTeams.csv']
Loaded and converted to SQL: Cities.csv 

# Submission Format

In [49]:
# Show all columns
pd.set_option('display.max_columns', None)

query = """
select 
*
from WNCAATourneySeeds

UNION ALL

select
*
from MNCAATourneySeeds
"""
seed_df = query_to_dataframe(query)
seed_df.head()


,Season,Seed,TeamID
0,1998,W01,3330
1,1998,W02,3163
2,1998,W03,3112
3,1998,W04,3301
4,1998,W05,3272


In [50]:
query = """
select 
*
from SampleSubmissionStage1
"""
submission_df = query_to_dataframe(query)
submission_df.head()

,ID,Pred
0,2021_1101_1102,0.5
1,2021_1101_1103,0.5
2,2021_1101_1104,0.5
3,2021_1101_1105,0.5
4,2021_1101_1106,0.5


In [51]:
def extract_game_info(id_str):
    # Extract year and team_ids
    parts = id_str.split('_')
    year = int(parts[0])
    teamID1 = int(parts[1])
    teamID2 = int(parts[2])
    return year, teamID1, teamID2

def extract_seed_value(seed_str):
    # Extract seed value
    try:
        return int(seed_str[1:])
    # Set seed to 16 for unselected teams and errors
    except ValueError:
        return 16

# Reformat the data
submission_df[['Season', 'TeamID1', 'TeamID2']] = submission_df['ID'].apply(extract_game_info).tolist()
seed_df['SeedValue'] = seed_df['Seed'].apply(extract_seed_value)

# Merge seed information for TeamID1
submission_df = pd.merge(submission_df, seed_df[['Season', 'TeamID', 'SeedValue']],
                         left_on=['Season', 'TeamID1'], right_on=['Season', 'TeamID'],
                         how='left')
submission_df = submission_df.rename(columns={'SeedValue': 'SeedValue1'}).drop(columns=['TeamID'])

# Merge seed information for TeamID2
submission_df = pd.merge(submission_df, seed_df[['Season', 'TeamID', 'SeedValue']],
                         left_on=['Season', 'TeamID2'], right_on=['Season', 'TeamID'],
                         how='left')
submission_df = submission_df.rename(columns={'SeedValue': 'SeedValue2'}).drop(columns=['TeamID'])
submission_df.head()

,ID,Pred,Season,TeamID1,TeamID2,SeedValue1,SeedValue2
0,2021_1101_1102,0.5,2021,1101,1102,14.0,NaN
1,2021_1101_1103,0.5,2021,1101,1103,14.0,NaN
2,2021_1101_1104,0.5,2021,1101,1104,14.0,2.0
3,2021_1101_1105,0.5,2021,1101,1105,14.0,NaN
4,2021_1101_1106,0.5,2021,1101,1106,14.0,NaN


In [53]:
# Calculate seed difference
submission_df['SeedDiff'] = submission_df['SeedValue1'] - submission_df['SeedValue2']

# Update 'Pred' column
submission_df['Pred'] = 0.5 + (0.03 * submission_df['SeedDiff'])

# Drop unnecessary columns
submission_df = submission_df[['ID', 'Pred']].fillna(0.5)

# Preview your submission 
submission_df.head()

,ID,Pred
0,2021_1101_1102,0.50
1,2021_1101_1103,0.50
2,2021_1101_1104,0.86
3,2021_1101_1105,0.50
4,2021_1101_1106,0.50


In [54]:
submission_df.to_csv('submission.csv', index=False)

# Feature Selection

As with prior years, each game has a unique ID created by concatenating the season in which the game was played and the two team's respective TeamIds. For example, "2025_1101_1102" indicates a hypothetical matchup between team 1101 and 1102 in the year 2025. You must predict the probability that the team with the lower TeamId beats the team with the higher TeamId. Note that the men's teams and women's TeamIds do not overlap.


In [81]:
# Show all columns
pd.set_option('display.max_columns', None)

query = """
select * from WRegularSeasonDetailedResults

UNION ALL
select * from WRegularSeasonDetailedResults



Limit 5
"""
df = pd.read_sql_query(query,engine)
table_name ='AllRegularSeasonResults'
df.to_sql(table_name, engine, if_exists='replace', index=False)



5

In [82]:
# Show all columns
pd.set_option('display.max_columns', None)

query = """
select
*
from AllRegularSeasonResults

Limit 5
"""
pd.read_sql_query(query,engine)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,11,3103,63,3237,49,H,0,23,54,5,9,12,19,10,26,14,18,7,0,15,20,54,3,13,6,10,11,27,11,23,7,6,19
1,2010,11,3104,73,3399,68,N,0,26,62,5,12,16,28,16,31,15,20,5,2,25,25,63,4,21,14,27,14,26,7,20,4,2,27
2,2010,11,3110,71,3224,59,A,0,29,62,6,15,7,12,14,23,18,13,6,2,17,19,58,2,14,19,23,17,23,8,15,6,0,15
3,2010,11,3111,63,3267,58,A,0,27,52,4,11,5,9,6,40,14,27,5,10,18,18,74,6,26,16,25,22,22,15,11,14,5,14
4,2010,11,3119,74,3447,70,H,1,30,74,7,20,7,11,14,33,18,11,5,3,18,25,74,9,17,11,21,21,32,12,14,4,2,14


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Create features from the statistics
def create_features(df):
    features = pd.DataFrame()
    
    # Replace infinities and NaN
    features = features.replace([np.inf, -np.inf], np.nan)
    features = features.fillna(0)
    
    return features

# Create target (point difference)
X = create_features(df)
y = df['WScore'] - df['LScore']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
